In [5]:
import os
os.chdir('CS687/HW1_TimeSeries')
os.listdir()

['cs674_hw1(1).pdf',
 'dtw.py',
 'dtw_playing_around.ipynb',
 'hw1_datasets',
 'hw1_main.py',
 'process_data.py',
 'testing_code.ipynb',
 'test_output',
 'timeseries_kNN.py',
 '__pycache__']

In [6]:
# os.chdir('CS687/HW1_TimeSeries')
# os.listdir()

['cs674_hw1(1).pdf',
 'dtw.py',
 'dtw_playing_around.ipynb',
 'hw1_datasets',
 'hw1_main.py',
 'process_data.py',
 'testing_code.ipynb',
 'test_output',
 'timeseries_kNN.py',
 '__pycache__']

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from timeseries_kNN import timeseries_kNN
import os
from heapq import nsmallest
from random import shuffle
from datetime import datetime as dt
from multiprocessing.pool import ThreadPool
from multiprocessing import Pool, cpu_count
from threading import Thread
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from dask import delayed
from datetime import datetime as dt

In [8]:
dataset1_test = 'hw1_datasets/dataset1/test_normalized.csv'
test1_labels = 'hw1_datasets/dataset1/test_labels.csv'
dataset1_train = 'hw1_datasets/dataset1/train_normalized.csv'
train1_labels = 'hw1_datasets/dataset1/train_labels.csv'
train1 = pd.read_csv(dataset1_train)
test1 = pd.read_csv(dataset1_test, index_col=0)
train1.head()
test1.head()

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,-1.186546,1.112753,1.499844,1.081663,1.425415,2.153796,1.518677,0.817144,1.196449,1.907417,...,0.920101,-0.007463,-0.218600,-0.494948,-0.103557,0.664229,0.945652,1.731896,1.228174,0.475412
1,0.407897,-0.583292,-0.739038,-1.673533,-0.065568,-0.018168,0.915209,-0.453858,-0.679295,-0.213708,...,-0.793742,-0.439977,-1.293828,1.242226,-0.620446,-0.492400,0.529914,-0.189582,0.256176,1.552341
2,-0.980611,-0.351831,-0.501440,-0.249934,-0.965480,-0.124003,-0.361499,-0.213201,-0.715557,-0.190532,...,-1.147009,-0.598236,-1.007310,-0.580817,-0.410211,-0.869793,-0.745043,-0.487621,-0.930685,-1.047306
3,-0.085029,0.091052,-1.140404,0.260809,0.360512,0.342949,-0.849694,-1.344433,-1.729363,-0.365701,...,-0.732892,0.455786,-0.905818,-0.034224,0.170775,0.470925,-0.173845,-0.166626,0.170123,0.135435
4,-1.053011,0.433085,-0.453950,0.146030,0.326497,-0.988377,-0.666821,-0.566277,-1.891093,-0.348393,...,3.153259,3.073105,3.398803,3.306304,4.264531,0.038569,-0.217056,-1.153955,-1.066324,-1.698859


In [9]:
x = np.array(range(0,10))
y = np.array(range(0,10))
len(x)

10

In [10]:
x.shape == y.shape

True

In [11]:
def get_time():
    time = dt.now()
    hour, minute = str(time.hour), str(time.minute)
    if(len(minute) == 1):
        minute = '0'+ minute
    if(len(hour) == 1):
        hour = '0' + hour
    if (len(second) ==1):
        second = '0' + second
    time = hour + minute + '.' + second
    return time

In [12]:
def dtw_dist(verbose, x, y, w):
    if verbose:
        print('in dtw_dist: {}'.format(get_time()))
    # pass
    width = w
    # calculate distance matrix
    dtw_dist_mx = create_dist_mx(verbose, width, x, y)
    # calculate shortest path

    return shortest_path

In [13]:
def calculate_width(verbose):
    if verbose:
        print("Calculating width: {}".format(get_time()))
    pass

In [14]:
def create_dist_mx(verbose, x, y):
    if verbose:
        print('Creating distance Matrix: {}'.format(get_time()))
    #create an array
    assert x.shape == y.shape, 'The shapes dont match'
    dist_mx = np.zeros(shape=(x.shape[0], y.shape[0]))
    dist_mx[:] = np.inf
    i = 0
    j = 0
    for i in range(0, dist_mx.shape[0]):
        for j in range(0, dist_mx.shape[0]):
            dist_mx[i][j] = np.abs(x[i] - y[j])
    return dist_mx

In [15]:
def shortest_path(verbose, dist_arr, width):
    if verbose:
        print("Calculating shortest distance; {}".format(get_time()))
    # should i start at the end or begining
    i, j, iteration = 0, 0, 0
    # when the final j is reached, the path is found
    last_i, last_j = dist_arr.shape[0], dist_arr.shape[1]
    
    path = [] # keep each index, may not be necessary
    path_cost = [] # keep cost of each step
    still_searching = True
    while(still_searching): # maybe lastj - w and lasti - w
        if verbose: print('Searching for shortest path index i: {};\tindex j: {}'.format(i,j))
        step, i, j = get_next_step(i, j, dist_arr, width, iteration) # [index, value] is this a problem? The distance matrix will be in memory now 3 times? or am i just passing by reference
        path.append([i, j])
        path_cost.append(step[1])
        iteration += 1
        
        if (i >= last_i) or (j >= last_j):  # might change to include width
            still_searching = False
            
    return np.sum(path_cost), path, iteration

In [16]:
mx = create_dist_mx(False, x, y)

In [17]:
mx

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [ 1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
       [ 4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.,  5.],
       [ 5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.,  4.],
       [ 6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.,  3.],
       [ 7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.,  2.],
       [ 8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.,  1.],
       [ 9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,  0.]])

In [52]:
def get_next_step(i, j, dist_mx, w, iteration):
    # make an assertion regarding size of i and j?
    if w is None:
        w = iteration
    
    # checking if parameters are within the warping width
    if i <= (iteration-w):
        diagonal, right, down = dist_mx[i+1][j+1], np.inf, dist_mx[i+1][j]
    elif j <= (iteration-w):
        diagonal, right, down = dist_mx[i+1][j+1], dist_mx[i][j+1], np.inf 
    else: 
        diagonal, right, down = dist_mx[i+1][j+1], dist_mx[i][j+1], dist_mx[i+1][j]

    step_mx = [diagonal, right, down]
    next_step = [np.argmin(step_mx), min(step_mx)]  # [index, value]
    
    if next_step[0] == 0: # column increases, step right
        j += 1
    elif next_step[0] == 1:  # diagonal step
        i += 1
        j += 1
    else: # step down
        i += 1
    return next_step, i, j

In [53]:
shortest_path(False, mx, None)

IndexError: index 10 is out of bounds for axis 0 with size 10

In [31]:
w= None

In [32]:
if w is None:
    print('hey')
else:
    print('else')

hey


In [25]:
a = [1,23,None,7,8,99,34]
min(a[0], a[4], a[6])

1

In [48]:
min(range(len(a)), key=a.__getitem__)

0

In [47]:
a.__getitem__(2)

5

In [27]:
np.argmin(a)
# np.min(a)

TypeError: unorderable types: NoneType() < int()

In [19]:
a = [3, 4, 5]
a

[3, 4, 5]

In [60]:
7mn = [[1,2,3],[4,5,7]]
# id = [1,2]
# id = slice(0,1)
id = '1,2'
mn[eval(id)]
# mn[id]

TypeError: list indices must be integers or slices, not tuple